<a href="https://colab.research.google.com/github/macronald88/ai-audit-assistant/blob/main/BankAuditing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0502_Data Consultants & Architects**

**Use Case: AI-Powered Expert System for Automated Audit Assistance**

**Problem Statement**

Auditors spend a large amount of time manually checking documents, matching records, and writing reports. These tasks are repetitive, error-prone, and slow, especially when dealing with high volumes of data. There is a need for a smart system that can analyze and explain audit data quickly and accurately

# SET-UP

In [28]:
!pip install pandas openpyxl openai
!pip install -q streamlit
!pip install openai
!pip install pandas openpyxl openai
!pip install streamlit pandas

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import openai
import io


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# AI Model Development (with Deep Learning and LLM)

# Source Files

1.Bank statement -Generated by the Bank

2.General leddger - Generated from the Accounting system.It contains all transactions from 1 July - 31 December

3.Purchases Invoices - Generated from Accounting system(Detailed report by Suppliers)

4.Receipts -Generated from the Point of Sale System/manually captured receipts register(note the POS is usually separeted from the accounting system)

5.Sales - Generated from Accounting system(Detailed sales report by Customers)

6.Supplier statements-Generated from Accounting system(Detailed sales report by Customers)

#2. Data Loading and Preprocessing

In [ ]:
import pandas

bank = pandas.read_csv('/content/drive/MyDrive/BUDT751/DATASETS/bank_statement.csv')
ledger = pandas.read_csv('/content/drive/MyDrive/BUDT751/DATASETS/general_ledger.csv')
purchase = pandas.read_csv('/content/drive/MyDrive/BUDT751/DATASETS/purchase_invoices.csv')
receipts = pandas.read_csv('/content/drive/MyDrive/BUDT751/DATASETS/receipts.csv')
sales = pandas.read_csv('/content/drive/MyDrive/BUDT751/DATASETS/sales_invoices.csv')
suppliers = pandas.read_csv('/content/drive/MyDrive/BUDT751/DATASETS/supplier_statements.csv')

# AI Model Development (with Deep Learning and LLM)

In [24]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import openai
import io


# --- 4.1 Data Loading and Preprocessing ---
def load_data(file_path):
    """Loads data from a file-like object (like from st.file_uploader)."""
    try:
        # When using st.file_uploader, the file_path is a file-like object.
        # You need to read from this object, not treat it as a string path.
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        # Catch potential errors during file reading
        print(f"Error reading file: {e}")
        return None # Return None or raise a specific error


def preprocess_dates(df, date_columns):
    """Converts specified columns to datetime."""
    # Add a check if df is None in case load_data failed
    if df is None:
        print("Warning: Cannot preprocess dates, DataFrame is None.")
        return None
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
        else:
            print(f"Warning: Date column '{col}' not found.")
    return df


def clean_data(df):
    """Cleans data (drop duplicates, fill NaNs)."""
    # Add a check if df is None
    if df is None:
        print("Warning: Cannot clean data, DataFrame is None.")
        return None
    df = df.drop_duplicates()
    df = df.fillna(0)
    return df


# No need for prepare_data_for_dl and DL model functions
# since we are focusing on rule-based checks


# --- 4.3 LLM Integration for Explanation ---
def get_llm_explanation(anomaly_data, context_data, anomaly_type=""):
    """
    Gets an explanation for the detected anomalies from an LLM.
    """
    openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace with your key

    # Convert anomaly data to string
    # Add checks if anomaly_data or context_data are None or not DataFrames
    anomaly_data_str = ""
    if isinstance(anomaly_data, pd.DataFrame):
         anomaly_data_str = anomaly_data.to_string()

    context_data_str = ""
    if isinstance(context_data, pd.DataFrame):
        context_data_str = context_data.to_string()


    prompt = f"""
    You are an expert auditor.  The following data represents potentially anomalous
    financial transactions:

    Anomaly Type: {anomaly_type}

    Anomalous Data:
    {anomaly_data_str}

    Context Data:
    {context_data_str}

    Explain why these transactions might be anomalous, and what further
    investigation should be done.  Provide your explanation as a markdown list.
    """

    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",  # Or another suitable model
            prompt=prompt,
            max_tokens=500,
            temperature=0.5,  # Adjust for desired randomness
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error getting LLM explanation: {e}"


# --- 4.4 Main Analysis Function ---
def analyze_audit_data(bank_statement_file, supplier_statements_file,
                       purchase_invoices_file, sales_invoices_file, receipts_file): # Order changed
    """
    Main function to orchestrate the audit data analysis.
    1.  Loads and preprocesses data.
    2.  Performs rule-based checks for the specified audit issues.
    3.  Uses an LLM to explain any discrepancies.
    """
    try:
        # Load data using the load_data function, which should handle file-like objects
        bank_statement_df = load_data(bank_statement_file)
        supplier_statements_df = load_data(supplier_statements_file)
        purchase_invoices_df = load_data(purchase_invoices_file)
        sales_invoices_df = load_data(sales_invoices_file)
        receipts_df = load_data(receipts_file)  # Load receipts data
    except Exception as e:
        return {"error": f"Error loading files: {e}"}

    # Check if any DataFrame failed to load
    if any(df is None for df in [bank_statement_df, supplier_statements_df,
                                 purchase_invoices_df, sales_invoices_df, receipts_df]):
        return {"error": "One or more files failed to load."}

    # Preprocess dates
    bank_statement_df = preprocess_dates(bank_statement_df, ['Date'])
    supplier_statements_df = preprocess_dates(supplier_statements_df, ['Date'])
    purchase_invoices_df = preprocess_dates(purchase_invoices_df, ['Date'])
    sales_invoices_df = preprocess_dates(sales_invoices_df, ['Date'])
    receipts_df = preprocess_dates(receipts_df, ['Date'])

    # Clean data
    bank_statement_df = clean_data(bank_statement_df)
    supplier_statements_df = clean_data(supplier_statements_df)
    purchase_invoices_df = clean_data(purchase_invoices_df)
    sales_invoices_df = clean_data(sales_invoices_df)
    receipts_df = clean_data(receipts_df)

    # Check if any DataFrame became None after cleaning
    if any(df is None for df in [bank_statement_df, supplier_statements_df,
                                 purchase_invoices_df, sales_invoices_df, receipts_df]):
        return {"error": "Error during data cleaning."}


    # --- Perform Audit Checks ---
    # Ensure all required dataframes are not None before calling comparison functions
    comparison_results = {}
    if bank_statement_df is not None and supplier_statements_df is not None:
        comparison_results = compare_bank_statement_supplier(bank_statement_df, supplier_statements_df)

    missing_transactions_results = {}
    if purchase_invoices_df is not None and sales_invoices_df is not None and receipts_df is not None:
        missing_transactions_results = check_missing_transactions(purchase_invoices_df, sales_invoices_df, receipts_df)

    receipts_comparison_results = {}
    if bank_statement_df is not None and receipts_df is not None:
        receipts_comparison_results = compare_bank_statement_receipts(bank_statement_df, receipts_df)


    # --- Get LLM Explanations ---
    # Ensure dataframes for LLM are not None before concatenating
    supplier_anomaly_data = pd.DataFrame()
    if "supplier_payments_not_on_bank" in comparison_results and "bank_statement_payments_not_on_supplier" in comparison_results and "amount_differences" in comparison_results:
        supplier_anomaly_data = pd.concat([
            comparison_results["supplier_payments_not_on_bank"],
            comparison_results["bank_statement_payments_not_on_supplier"],
            comparison_results["amount_differences"]
        ])
    supplier_context_data = pd.DataFrame()
    if bank_statement_df is not None and supplier_statements_df is not None:
         supplier_context_data = pd.concat([bank_statement_df, supplier_statements_df])


    llm_explanation_supplier = get_llm_explanation(
        supplier_anomaly_data,
        supplier_context_data,
        "Bank vs. Supplier Statement Discrepancies"
    )

    missing_anomaly_data = pd.DataFrame()
    if "missing_purchase_invoices" in missing_transactions_results and "missing_sales_invoices" in missing_transactions_results and "missing_receipts" in missing_transactions_results:
        missing_anomaly_data = pd.DataFrame({
            "Missing Purchase Invoices": [len(missing_transactions_results["missing_purchase_invoices"])],
            "Missing Sales Invoices": [len(missing_transactions_results["missing_sales_invoices"])],
            "Missing Receipts": [len(missing_transactions_results["missing_receipts"])]
        })

    missing_context_data = pd.DataFrame()
    if purchase_invoices_df is not None and sales_invoices_df is not None and receipts_df is not None:
        missing_context_data = pd.concat([purchase_invoices_df, sales_invoices_df, receipts_df])


    llm_explanation_missing = get_llm_explanation(
        missing_anomaly_data,
        missing_context_data,
        "Missing Transactions"
    )

    receipts_anomaly_data = pd.DataFrame()
    if "receipts_not_on_bank" in receipts_comparison_results and "bank_statement_not_on_receipts" in receipts_comparison_results and "amount_differences" in receipts_comparison_results:
        receipts_anomaly_data = pd.concat([
            receipts_comparison_results["receipts_not_on_bank"],
            receipts_comparison_results["bank_statement_not_on_receipts"],
            receipts_comparison_results["amount_differences"]
        ])

    receipts_context_data = pd.DataFrame()
    if bank_statement_df is not None and receipts_df is not None:
        receipts_context_data = pd.concat([bank_statement_df, receipts_df])

    llm_explanation_receipts = get_llm_explanation(
        receipts_anomaly_data,
        receipts_context_data,
        "Bank vs. Receipts Discrepancies"
    )


    return {
        "bank_supplier_comparison": comparison_results,
        "missing_transactions": missing_transactions_results,
        "bank_receipts_comparison": receipts_comparison_results,
        "llm_explanation_supplier": llm_explanation_supplier,
        "llm_explanation_missing": llm_explanation_missing,
        "llm_explanation_receipts": llm_explanation_receipts
    }


# --- 4.5 Helper Functions (Rule-Based Checks) ---
def check_missing_transactions(purchase_invoices_df, sales_invoices_df, receipts_df):
    """Checks for missing transactions."""
    def find_missing(df, id_col):
        if df is None or id_col not in df.columns:
            return []
        # Ensure the ID column is numeric before sorting and finding min/max
        if not pd.api.types.is_numeric_dtype(df[id_col]):
             print(f"Warning: ID column '{id_col}' is not numeric. Skipping missing transaction check.")
             return []

        df = df.sort_values(by=id_col)
        if df[id_col].empty:
            return []
        # Handle potential non-integer IDs by converting to int for range creation
        try:
            min_id = int(df[id_col].min())
            max_id = int(df[id_col].max())
            expected_ids = set(range(min_id, max_id + 1))
            actual_ids = set(df[id_col].astype(int)) # Convert actual IDs to int as well
            missing_ids = sorted(list(expected_ids - actual_ids))
            return missing_ids
        except ValueError as e:
             print(f"Error converting ID column '{id_col}' to integer: {e}. Skipping missing transaction check.")
             return []


    missing_purchase_invoices = find_missing(purchase_invoices_df, 'Invoice Number')
    missing_sales_invoices = find_missing(sales_invoices_df, 'Invoice number')
    missing_receipts = find_missing(receipts_df, 'Receipt Number')

    return {
        "missing_purchase_invoices": missing_purchase_invoices,
        "missing_sales_invoices": missing_sales_invoices,
        "missing_receipts": missing_receipts
    }


def compare_bank_statement_receipts(bank_statement_df, receipts_df):
    """Compares bank statement and receipts."""
    if bank_statement_df is None or receipts_df is None:
        return {}
    # Ensure date columns are datetime objects before comparison
    bank_statement_df = preprocess_dates(bank_statement_df, ['Date'])
    receipts_df = preprocess_dates(receipts_df, ['Date'])

    # Drop rows where date conversion failed
    bank_statement_df = bank_statement_df.dropna(subset=['Date'])
    receipts_df = receipts_df.dropna(subset=['Date'])


    # Convert relevant columns to a consistent type for comparison (e.g., string or numeric)
    # Ensure 'Cr' and 'Amount' are numeric
    bank_statement_df['Cr'] = pd.to_numeric(bank_statement_df['Cr'], errors='coerce').fillna(0)
    receipts_df['Amount'] = pd.to_numeric(receipts_df['Amount'], errors='coerce').fillna(0)

    # Convert identifier columns to string to avoid type mismatches during comparison
    bank_statement_df['Detail'] = bank_statement_df['Detail'].astype(str)
    receipts_df['Invoice Number'] = receipts_df['Invoice Number'].astype(str)
    receipts_df['Receipt Number'] = receipts_df['Receipt Number'].astype(str)


    # More robust comparison logic
    # receipts not on bank: find rows in receipts_df that don't have a match in bank_statement_df
    # based on Date, Amount, and potentially Invoice Number (or Receipt Number depending on mapping)
    # This fuzzy matching with isin(df.to_dict('list')) is generally not reliable for comparisons
    # Consider merging or more explicit comparison logic.

    # Let's assume we want to match based on Date and Amount, and potentially Invoice/Receipt number as a detail
    # Create a unique identifier for merging/comparison if possible

    # Option 1: Merge and identify non-matches
    merged_receipts_bank = pd.merge(
        receipts_df,
        bank_statement_df,
        left_on=['Date', 'Amount'], # Assuming Amount in receipts matches Cr in bank
        right_on=['Date', 'Cr'],
        how='left',
        indicator=True
    )
    # receipts not found in bank
    receipts_not_on_bank = merged_receipts_bank[merged_receipts_bank['_merge'] == 'left_only'][receipts_df.columns] # Keep only receipts columns

    # bank statement not on receipts
    merged_bank_receipts = pd.merge(
        bank_statement_df,
        receipts_df,
        left_on=['Date', 'Cr'],
        right_on=['Date', 'Amount'],
        how='left',
        indicator=True
    )
    # bank entries not found in receipts
    bank_statement_not_on_receipts = merged_bank_receipts[merged_bank_receipts['_merge'] == 'left_only'][bank_statement_df.columns] # Keep only bank columns

    # Transactions with different amounts - Requires a common identifier other than just Date and Amount
    # If 'Detail' in bank_statement_df corresponds to 'Receipt Number' in receipts_df for receipts transactions:
    merged_amount_check = pd.merge(
        bank_statement_df,
        receipts_df,
        left_on=['Date', 'Detail'], # Assuming Detail maps to Receipt Number for receipts
        right_on=['Date', 'Receipt Number'],
        how='inner' # Only look at transactions present in both based on Date and identifier
    )

    # Filter for rows where amounts don't match
    amount_differences = merged_amount_check[merged_amount_check['Cr'] != merged_amount_check['Amount']]


    return {
        "receipts_not_on_bank": receipts_not_on_bank,
        "bank_statement_not_on_receipts": bank_statement_not_on_receipts,
        "amount_differences": amount_differences
    }


def compare_bank_statement_supplier(bank_statement_df, supplier_statements_df):
    """Compares bank statement and supplier statements."""
    if bank_statement_df is None or supplier_statements_df is None:
        return {}

    # Ensure date columns are datetime objects before comparison
    bank_statement_df = preprocess_dates(bank_statement_df, ['Date'])
    supplier_statements_df = preprocess_dates(supplier_statements_df, ['Date'])

    # Drop rows where date conversion failed
    bank_statement_df = bank_statement_df.dropna(subset=['Date'])
    supplier_statements_df = supplier_statements_df.dropna(subset=['Date'])

    # Ensure relevant columns are numeric
    bank_statement_df['Cr'] = pd.to_numeric(bank_statement_df['Cr'], errors='coerce').fillna(0)
    supplier_statements_df['Credit (Payments/Credit Notes Applied)'] = pd.to_numeric(supplier_statements_df['Credit (Payments/Credit Notes Applied)'], errors='coerce').fillna(0)

    # Ensure identifier columns are string
    bank_statement_df['Detail'] = bank_statement_df['Detail'].astype(str)
    supplier_statements_df['Payment Ref #Invoice number'] = supplier_statements_df['Payment Ref #Invoice number'].astype(str)


    # supplier payments not on bank
    merged_supplier_bank = pd.merge(
        supplier_statements_df,
        bank_statement_df,
        left_on=['Date', 'Credit (Payments/Credit Notes Applied)'], # Assuming Payment Amount matches Cr in bank
        right_on=['Date', 'Cr'],
        how='left',
        indicator=True
    )
    supplier_payments_not_on_bank = merged_supplier_bank[merged_supplier_bank['_merge'] == 'left_only'][supplier_statements_df.columns] # Keep supplier columns

    # bank statement payments not on supplier
    merged_bank_supplier = pd.merge(
        bank_statement_df,
        supplier_statements_df,
        left_on=['Date', 'Cr'],
        right_on=['Date', 'Credit (Payments/Credit Notes Applied)'],
        how='left',
        indicator=True
    )
    bank_statement_payments_not_on_supplier = merged_bank_supplier[merged_bank_supplier['_merge'] == 'left_only'][bank_statement_df.columns] # Keep bank columns


    # Transactions with different amounts - Requires a common identifier
    # If 'Detail' in bank_statement_df corresponds to 'Payment Ref #Invoice number' in supplier_statements_df for payment transactions:
    merged_amount_diff = pd.merge(
        bank_statement_df,
        supplier_statements_df,
        left_on=['Date', 'Detail'], # Assuming Detail maps to Payment Ref #Invoice number
        right_on=['Date', 'Payment Ref #Invoice number'],
        how='inner'
    )

    amount_differences = merged_amount_diff[
        merged_amount_diff['Cr'] != merged_amount_diff['Credit (Payments/Credit Notes Applied)']
    ]


    return {
        "supplier_payments_not_on_bank": supplier_payments_not_on_bank,
        "bank_statement_payments_not_on_supplier": bank_statement_payments_not_on_supplier,
        "amount_differences": amount_differences
    }

# --- 5. Step 5: Integrating with Streamlit ---
# This function is intended for use in a Streamlit app (app.py)
# It should not be in a Jupyter notebook cell that's executed sequentially
# without the Streamlit context.
# To prevent errors, this function and the related Streamlit UI code below it
# should be moved to your app.py file.

# def display_audit_results(analysis_results):
#     """Displays the audit analysis results in Streamlit."""
#     # Note: This function is intended to be used within a Streamlit application
#     # where 'st' is imported. The code below assumes 'st' is available in that context.

#     # Import streamlit if running this function outside of a full Streamlit app
#     # try:
#     #     import streamlit as st
#     # except ImportError:
#     #     print("Streamlit is not installed. Please install it to use this function.")
#     #     return

#     st.subheader("Audit Analysis Results")

#     if "error" in analysis_results:
#         st.error(analysis_results["error"])
#         return

#     st.subheader("Bank Statement vs. Supplier Statements Comparison")
#     st.write(analysis_results["llm_explanation_supplier"])
#     st.write("Supplier payments not on bank statement",analysis_results["bank_supplier_comparison"]["supplier_payments_not_on_bank"])
#     st.write("Bank statement payments not on supplier statements",analysis_results["bank_supplier_comparison"]["bank_statement_payments_not_on_supplier"])
#     st.write("Transactions with different amounts",analysis_results["bank_supplier_comparison"]["amount_differences"])

#     st.subheader("Missing Transaction Checks")
#     st.write(analysis_results["llm_explanation_missing"])
#     st.write("Missing Purchase Invoices:", analysis_results["missing_transactions"]["missing_purchase_invoices"])
#     st.write("Missing Sales Invoices:", analysis_results["missing_transactions"]["missing_sales_invoices"])
#     st.write("Missing Receipts:", analysis_results["missing_transactions"]["missing_receipts"])

#     st.subheader("Bank Statement vs. Receipts Comparison")
#     st.write(analysis_results["llm_explanation_receipts"])
#     st.write("Receipts not on bank statement",analysis_results["bank_receipts_comparison"]["receipts_not_on_bank"])
#     st.write("Transactions only in Bank Statement:", analysis_results["bank_receipts_comparison"]["bank_statement_not_on_receipts"])
#     st.write("Transactions with different amounts:", analysis_results["bank_receipts_comparison"]["amount_differences"])

# The Streamlit file uploading and button logic below should be in your separate Streamlit application file (e.g., app.py)
# and not in this Jupyter notebook cell. The lines below are commented out to prevent the IndentationError in the notebook.

# # Store the uploaded files in session state and use them later
# if 'bank_statement_file' not in st.session_state:
#   st.session_state.bank_statement_file = None
# if 'receipts_file' not in st.session_state:
#   st.session_state.receipts_file = None
# if 'supplier_statements_file' not in st.session_state:
#   st.session_state.supplier_statements_file = None
# if 'purchase_invoices_file' not in st.session_state:
#   st.session_state.purchase_invoices_file = None
# if 'sales_invoices_file' not in st.session_state:
#   st.session_state.sales_invoices_file = None

# # ... (Streamlit code from Step 2 - app.py)
# elif choice == "Audit Analysis":
#   st.subheader("Upload Financial Data")
#   st.write("Upload CSV files for AI-powered audit analysis.")

#   bank_statement_file = st.file_uploader("Bank Statement (CSV)", type="csv")
#   receipts_file = st.file_uploader("Receipts (CSV)", type="csv")
#   supplier_statements_file = st.file_uploader("Supplier Statements (CSV)", type="csv")
#   purchase_invoices_file = st.file_uploader("Purchase Invoices (CSV)", type="csv")
#   sales_invoices_file = st.file_uploader("Sales Invoices (CSV)", type="csv")

#   if bank_statement_file and receipts_file and supplier_statements_file and purchase_invoices_file and sales_invoices_file:
#   st.success("Files uploaded. Click 'Analyze' to start AI audit analysis.")

#   # Store the file in session state
#   st.session_state.bank_statement_file = bank_statement_file
#   st.session_state.receipts_file = receipts_file
#   st.session_state.supplier_statements_file = supplier_statements_file
#   st.session_state.purchase_invoices_file = purchase_invoices_file
#   st.session_state.sales_invoices_file = sales_invoices_file

#   if st.button("Analyze"):
#   st.write("Running AI-powered analysis...")
#   analysis_results = analyze_audit_data(
#   bank_statement_file, supplier_statements_file,
#   purchase_invoices_file, sales_invoices_file, receipts_file #order changed
#   )
#   display_audit_results(analysis_results)

# # Example Usage (for testing in Colab)
# #if __name__ == '__main__':
# #  #  Create sample DataFrames (replace with your actual data)
# #  bank_statement_data = {'Date': ['2024-01-01', '2024-01-05', '2024-01-10', '2024-01-15', '2024-01-20'],
# #  'Detail': ['Payment', 'Receipt', 'Payment', 'Anomaly', 'Receipt'],
# #  'Customer/Supplier name' : ['A','B','C','D','E'],
# #  'Cr': [100, 200, 150, 1000, 250],
# #  'Balance': [1000, 1200, 1050, 50, 300]}
# #  receipts_data = {'Date': ['2024-01-05', '2024-01-10', '2024-01-12'],
# #  'Receipt Number': [101, 102, 103],
# #  'Customer Name' : ['B','C','F'],
# #  'Invoice Number' : [1,2,3],
# #  'Amount': [200, 150, 250]}
# #  supplier_statements_data = {'Date': ['2024-01-01', '2024-01-1

# Reconciliation Logic

The goal is to detect mismatched transactions, which is important for account reconciliation and fraud detection. For example, if I pay UMD tuition, the transaction appears on the UMD bank statement but is not reflected in my student billing account. This project will examine:

Matching transaction records from different sources, such as:

Bank statements vs. receipts

Bank statements vs. supplier statements

General ledger vs. purchase invoices, sales invoices, and supplier statements

Missing transaction sequences in supplier invoices, customer invoices, and receipts.

**QUERIES**

##1. Bank statement vs Receipts

**To compare:**

1.a) Which transactions were recorded as received on receipts file, yet the receipt does not appear on the bank statement?

1.b) Which payments are on the bank statement but do not appear on the receipts file ?

1.c)Which transactions appear on both the bank statement and receipts file but have different amounts



 2. Bank statements vs Supplier statements

 To compare:
 2.a) Which transactions were recorded as a payment in supplier statements file , yet the rec does not appear on the bank statement?

# GPT EXPLANATION

In [ ]:
openai.api_key = "YOUR_OPENAI_API_KEY"  # Replace this

In [ ]:
def explain(row):
    prompt = f"""Audit Issue:\n\nSource: {row['source']}\nType: {row['issue_type']}\nDetails: {row['details']}\n\nPlease explain in plain English why this is an audit concern."""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"GPT error: {e}"

In [ ]:
# Turn into DataFrame
df_issues = pd.DataFrame(issues)
df_issues['explanation'] = df_issues.apply(explain, axis=1)

KeyError: 'source'

# Step 2 .Implementing the Landing Page and UI (Streamlit)

In [26]:
%%writefile app.py
import streamlit as st
import pandas as pd
import openai
import subprocess
import os

# --- Helper function for styling ---
def local_css(file_name):
    with open(file_name) as f:
        st.markdown(f"<style>{f.read()}</style>", unsafe_allow_html=True)

# Check if style.css exists, if not, create a minimal one
if not os.path.exists("style.css"):
    with open("style.css", "w") as f:
        f.write("""
body {
 font-family: Arial, sans-serif;
 color: #333;
}
.st-header {
 background-color: #f0f2f6;
 padding: 10px;
 border-bottom: 1px solid #ccc;
}
.st-subheader {
 color: #444;
}
""")

local_css("style.css")

# --- Header Section ---
st.header("AI-Powered Audit Assistant")
st.subheader("Automate reconciliation and discrepancy detection with AI.")

# --- Navigation ---
menu = ["Home", "Audit Analysis", "About Us", "Contact"]
choice = st.sidebar.selectbox("Menu", menu)

if choice == "Home":
    st.subheader("Streamline Your Audit Workflow")
    st.write("""
    - Automate time-consuming reconciliation.
    - Detect discrepancies with high accuracy using AI.
    - Get expert-level insights and explanations.
    """)
    st.subheader("Key Audit Analysis Features")
    st.write("• AI-Powered Discrepancy Detection")
    st.write("• Bank Statement vs. Receipts")
    st.write("• Bank Statement vs. Supplier Statements")
    st.write("• Missing Transactions")
    st.subheader("Request a Demo")
    st.button("Schedule a Demo")

elif choice == "Audit Analysis":
    st.subheader("Upload Financial Data")
    st.write("Upload CSV files for AI-powered audit analysis.")

    bank_statement_file = st.file_uploader("Bank Statement (CSV)", type="csv")
    receipts_file = st.file_uploader("Receipts (CSV)", type="csv")
    supplier_statements_file = st.file_uploader("Supplier Statements (CSV)", type="csv")
    purchase_invoices_file = st.file_uploader("Purchase Invoices (CSV)", type="csv")
    sales_invoices_file = st.file_uploader("Sales Invoices (CSV)", type="csv")

    if bank_statement_file and receipts_file and supplier_statements_file and purchase_invoices_file and sales_invoices_file:
        st.success("Files uploaded. Click 'Analyze' to start AI audit analysis.")
        if st.button("Analyze"):
            st.write("Running AI-powered analysis...")
            # Dummy placeholder
            st.info("Analysis complete. Display results here.")

elif choice == "About Us":
    st.subheader("About Data Consultants & Architects")
    st.write("AI-powered solutions for finance.")

elif choice == "Contact":
    st.subheader("Contact Us")
    st.write("Get in touch.")


Writing app.py


In [27]:
!pip install streamlit
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 3s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [ ]:
# Run Streamlit app in the background
import os
import subprocess

# Start streamlit in background
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Expose port 8501 with localtunnel
!npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴your url is: https://light-guests-mate.loca.lt
^C


# Step 3: GitHub Integration